In [ ]:
include("_.jl")

In [ ]:
me_ge_ = OrderedDict{String,Vector{String}}()

In [ ]:
en = read_ensembl(; or = "mouse")

PR_ = sort(string.(unique((en[en[!, "Gene type"].=="protein_coding", "Gene name"]))))

length(PR_)

## Ensembl

In [ ]:
or = TableAccess.read(joinpath(PAI, "ensembl", "mouse_to_human.tsv.gz"))

#or[!, "Gene name"] = rename(or[!, "Gene name"]))

;

In [ ]:
mo_hu = Dict{String,String}()

for ro in eachrow(or)

    mo = ro["Gene name"]

    hu = ro["Human gene name"]

    if !(ismissing(mo) || ismissing(hu))

        mo_hu[mo] = hu

    end

end

println(length(mo_hu))

me_ge_["Ensembl"] = [pr for pr in PR_ if !haskey(mo_hu, pr)]

## Mouse Genome Informatics

In [ ]:
mg = TableAccess.read(joinpath(PAI, "HOM_MouseHumanSequence.rpt.txt.gz"))

se_ = mg[!, "Common Organism Name"] .== "mouse, laboratory"

#mg[se_, "Symbol"] .= rename(string.(mg[se_, "Symbol"]))

;

In [ ]:
mo_hu = Dict{String,String}()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    se_ = da[!, "Common Organism Name"] .== "mouse, laboratory"

    for mo in sy_[se_], hu in sy_[.!se_]

        mo_hu[mo] = hu

    end

end

println(length(mo_hu))

me_ge_["MGI"] = [pr for pr in PR_ if !haskey(mo_hu, pr)]

## Write

In [ ]:
ge_ = unique(vcat(values(me_ge_)...))

an_ge_me = DataFrame("Gene" => ge_)

for (me, se_) in me_ge_

    an_ge_me[!, me] = [ge in se_ for ge in ge_]

end

an_ge_me[!, "Sum"] = vec(sum(Matrix(an_ge_me[!, 2:end]); dims = 2))

sort!(an_ge_me, "Sum"; rev = true)

TableAccess.write(joinpath(PAO, "mouse_gene_by_method.tsv"), an_ge_me)

an_ge_me